In [1]:
# Ensure script runs in the correct environment
cd(@__DIR__)
using Pkg
Pkg.activate("../.")
Pkg.resolve()
Pkg.instantiate()

# Import libraries
using DifferentialEquations
using Plots
using LinearAlgebra
using PowerSystems
using PowerFlows
const PSY = PowerSystems
const PF = PowerFlows


  Activating project at `~/projects/ecen_5447_scripts/ECEN-5447-assignments`
  No Changes to `~/projects/ecen_5447_scripts/ECEN-5447-assignments/Project.toml`
  No Changes to `~/projects/ecen_5447_scripts/ECEN-5447-assignments/Manifest.toml`


PowerFlows

Start with Power Flow Solution

In [2]:
sys = PSY.System("../data/ThreeBusMultiLoad.raw")
pf_result = PF.solve_powerflow(ACPowerFlow{}(), sys)

┌ Info: Did not find a PowerFlows.jl PSS/E export metadata file at ../data/ThreeBusMultiLoad_export_metadata.json, will not do any remapping
└ @ PowerSystems /Users/robstallman/.julia/packages/PowerSystems/azBUB/src/parsers/psse_metadata_reimport.jl:166
┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /Users/robstallman/.julia/packages/PowerSystems/azBUB/src/parsers/pm_io/common.jl:42
┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /Users/robstallman/.julia/packages/PowerSystems/azBUB/src/parsers/pm_io/psse.jl:1103
┌ Info: Parsing PSS(R)E Bus data into a PowerModels Dict...
└ @ PowerSystems /Users/robstallman/.julia/packages/PowerSystems/azBUB/src/parsers/pm_io/psse.jl:275
┌ Info: Parsing PSS(R)E Load data into a PowerModels Dict...
└ @ PowerSystems /Users/robstallman/.julia/packages/PowerSystems/azBUB/src/parsers/pm_

Dict{String, DataFrames.DataFrame} with 2 entries:
  "flow_results" => 3×9 DataFrame…
  "bus_results"  => 3×9 DataFrame…

In [3]:
# Get power flow output
v = pf_result["bus_results"].Vm     # [pu-V]
θ = pf_result["bus_results"].θ      # [rad]

# Sienna exports power injections in MW/MVar, so adjust by system base
P = pf_result["bus_results"].P_net / PSY.get_base_power(sys) # [pu(MW)]
Q = pf_result["bus_results"].Q_net / PSY.get_base_power(sys) # [pu(MVar)]

3-element Vector{Float64}:
  0.24501857920705117
  0.10751144041475172
 -0.3

In [4]:
print(v)

[1.02, 1.0142, 0.9869282927720892]

In [5]:
θ

3-element Vector{Float64}:
  0.0
 -0.007843738409384567
 -0.1104518610357432

In [6]:
P

3-element Vector{Float64}:
  1.017056789531292
  0.8
 -1.8

In [7]:
Q

3-element Vector{Float64}:
  0.24501857920705117
  0.10751144041475172
 -0.3

In [9]:
(V_0, θ_0, P_0, Q_0) = v[2], θ[2], P[2], Q[2]


(1.0142, -0.007843738409384567, 0.8, 0.10751144041475172)

In [11]:
# Terminal voltage
v = V_0 * ℯ^(im * θ_0)

1.0141688012217909 - 0.007955037922945551im

In [12]:
# Calculate initial current injection
i = conj(complex(P_0, Q_0)) / conj(v)

0.7879433124292734 - 0.11218996207454346im

In [13]:
v * conj(i)

0.8 + 0.10751144041475172im

In [15]:
# Machine Parameters (per unit)
struct MachineParams
    H::Float64      # Inertia constant (s)
    D::Float64      # Damping coefficient
    Xd::Float64     # d-axis synchronous reactance
    Xq::Float64     # q-axis synchronous reactance
    Xdp::Float64    # d-axis transient reactance
    Xqp::Float64    # q-axis transient reactance
    Xdpp::Float64   # d-axis subtransient reactance
    Xqpp::Float64   # q-axis subtransient reactance
    Xl::Float64     # Leakage reactance
    Td0p::Float64   # d-axis transient open-circuit time constant (s)
    Tq0p::Float64   # q-axis transient open-circuit time constant (s)
    Td0pp::Float64  # d-axis subtransient open-circuit time constant (s)
    Tq0pp::Float64  # q-axis subtransient open-circuit time constant (s)
    Taa::Float64    # d-axis additional leakage time constant (s) –– Presently unused
    Ra::Float64     # Armature resistance
end

In [16]:
function default_machine_params()
    return MachineParams(
        3.148,  # H
        2.0,    # D
        1.79,   # Xd
        1.71,   # Xq
        0.169,  # Xdp
        0.228,  # Xqp
        0.135,  # Xdpp
        0.2,    # Xqpp
        0.13,   # Xl
        4.3,    # Td0p
        0.85,   # Tq0p
        0.032,  # Td0pp
        0.05,   # Tq0pp
        0.00,   # Taa
        0.002   # Ra
    )
end
machine = default_machine_params()

MachineParams(3.148, 2.0, 1.79, 1.71, 0.169, 0.228, 0.135, 0.2, 0.13, 4.3, 0.85, 0.032, 0.05, 0.0, 0.002)

In [17]:
# Calculate initial rotor angle
δ0 = angle(v + complex(machine.Ra, machine.Xq) * i)             # Eqn. 9.11

0.8370306034067273

In [18]:
θ_0

-0.007843738409384567

In [19]:
complex(machine.Ra, machine.Xq) * i

0.19342072177232786 + 1.3471586843299084im

In [20]:
angle(complex(machine.Ra, machine.Xq) * i)

1.4281940875147179

In [21]:
rad2deg(angle(complex(machine.Ra, machine.Xq) * i))

81.82949354013107

In [22]:
rad2deg(θ_0)


-0.44941310646239324

In [23]:
complex(machine.Ra, machine.Xq)


0.002 + 1.71im

In [24]:
rad2deg(δ0)

47.9583208984941

In [25]:
vdq = v * ℯ^(-1 * im * (δ0 - π / 2))
Vd_0 = real(vdq)
Vq_0 = imag(vdq)

0.6732515847143696

In [26]:
V_0 * sin(δ0 - θ_0) - Vd_0

0.0

In [27]:
V_0 * cos(δ0 - θ_0) - Vq_0

0.0

In [28]:
# Current
idq = i * ℯ^(-1 * im * (δ0 - π / 2))
Id_0 = real(idq)
Iq_0 = imag(idq)

0.44434404311986414

In [29]:
# Prepare Milano's helper variables for the next step           # Eqn. 15.14
γd1 = (machine.Xdpp - machine.Xl) / (machine.Xdp - machine.Xl)
γq1 = (machine.Xqpp - machine.Xl) / (machine.Xqp - machine.Xl)

0.7142857142857143

In [30]:
A = [-γq1 0 0 (1-γq1) 0;
    0 γd1 (1-γd1) 0 0;
    0 1 -1 0 0;
    -1 0 0 -1 0;
    0 -1 0 0 1]
b = [machine.Xqpp * Iq_0 - machine.Ra * Id_0 - Vd_0;
    machine.Ra * Iq_0 + machine.Xdpp * Id_0 + Vq_0;
    (machine.Xdp - machine.Xl) * Id_0;
    (machine.Xqp - machine.Xl) * Iq_0;
    (machine.Xd - machine.Xdp) * Id_0]

(Ed_p0, Eq_p0, ψd_pp0, ψq_pp0, Vf_0) = A \ b

5-element Vector{Float64}:
  0.6585178719036389
  0.7857314248658887
  0.7599796205431318
 -0.7020635881293855
  1.8560820609476498

In [33]:
# Gut check
ψd_pp0_check = machine.Ra * Iq_0 + machine.Xdpp * Id_0 + Vq_0 - γd1 * (machine.Xdp - machine.Xl) * Id_0

0.7599796205431318

In [34]:
u0_SM = [δ0, 1.0, Eq_p0, Ed_p0, ψq_pp0, ψd_pp0]

6-element Vector{Float64}:
  0.8370306034067273
  1.0
  0.7857314248658887
  0.6585178719036389
 -0.7020635881293855
  0.7599796205431318

In [35]:
du0_SM = Vector{Float64}(undef, 6) # return object for derivative calculation

6-element Vector{Float64}:
 1.2541870494434764e219
 6.807402956849389e199
 9.701582850788981e189
 1.428816065556828e248
 6.815498219306276e246
 1.4288160655558137e248

In [38]:
p_SM = (machine, (V_0, θ_0, P_0, Q_0))

(MachineParams(3.148, 2.0, 1.79, 1.71, 0.169, 0.228, 0.135, 0.2, 0.13, 4.3, 0.85, 0.032, 0.05, 0.0, 0.002), (1.0142, -0.007843738409384567, 0.8, 0.10751144041475172))

In [40]:
function synchronous_machine_dynamics!(du, u, p, t)
    # Unpack parameters
    machine, pf_result = p
    (V, θ, P, Q) = pf_result

    # Unpack state variables
    δ, ω, Eq_p, Ed_p, ψq_pp, ψd_pp = u

    # Calculate terminal conditions
    # Stator voltage equations
    # TO-DO: CHECK WHETHER THE BUS ANGLE FROM POWER FLOW SOLUTION SHOULD BE INCLUDED
    Vd = V * sin(δ - θ)       # Eqn 15.4
    Vq = V * cos(δ - θ)       # Eqn 15.4
    println("Vd = $Vd\nVq = $Vq\n")

    # Terminal voltage
    Vt = sqrt(Vd^2 + Vq^2)

    # Calculate machine currents
    # Helper variables (Eqn 15.14)
    γd1 = (machine.Xdpp - machine.Xl) / (machine.Xdp - machine.Xl)
    γq1 = (machine.Xqpp - machine.Xl) / (machine.Xqp - machine.Xl)
    γd2 = (1 - γd1) / (machine.Xdp - machine.Xl)
    γq2 = (1 - γq1) / (machine.Xqp - machine.Xl)

    # Current equations (Use 15.11 and 15.15 to eliminate ψq and ψd, solve for Id and Iq)
    A = [machine.Xdpp machine.Ra; machine.Ra -machine.Xqpp]
    b = [-Vq + γd1 * Eq_p + (1 - γd1) * ψd_pp; -Vd + γq1 * Ed_p + (1 - γq1) * ψq_pp]
    currents = A \ b
    Id = currents[1]
    Iq = currents[2]
    println("Id = $Id\nIq = $Iq\n")

    # Calculate shaft conditions
    # Electromagnetic torque/power (Use 15.11 and 15.6 to eliminate ψq and ψd, solve for Te)
    Pe = Vd * Id + Vq * Iq + machine.Ra * (Id^2 + Iq^2)
    println("Pe = $Pe\n")
    # Te = Pe   # in per unit, torque equals power

    # Total mechanical power input
    Pm = Pe

    # Synchronous speed in pu
    ω_s = 1.0

    # Shaft mechanical equations (Eqn 15.5)
    du[1] = ω - ω_s  # dδ/dt
    du[2] = (Pm - Pe - machine.D * (ω - ω_s)) / (2.0 * machine.H)

    # Electrical dynamics
    du[3] = (-Eq_p - (machine.Xd - machine.Xdp) * (Id - γd2 * ψd_pp - (1 - γd1) * Id + γd2 * Eq_p) + Vf) / machine.Td0p    # dE'q/dt
    du[4] = (-Ed_p + (machine.Xq - machine.Xqp) * (Iq - γq2 * ψq_pp - (1 - γq1) * Iq - γq2 * Ed_p)) / machine.Tq0p         # dE'd/dt    TO-DO: CHECK TYPO IN γq2
    du[5] = (-ψd_pp + Eq_p - (machine.Xdp - machine.Xl) * Id) / machine.Td0pp                                              # dE''q/dt
    du[6] = (-ψq_pp - Ed_p - (machine.Xqp - machine.Xl) * Iq) / machine.Tq0pp                                              # dE''d/dt

    # Add numerical stability safeguards
    for i in 1:length(du)
        # Limit the rate of change to reasonable values
        if abs(du[i]) > 100.0
            du[i] = sign(du[i]) * 100.0
        end
    end
end

synchronous_machine_dynamics! (generic function with 1 method)

In [41]:
synchronous_machine_dynamics!(du0_SM, u0_SM, p_SM, 0)

Vd = 0.7585077083850829
Vq = 0.6732515847143696

Id = 0.6305900995301867
Iq = 2.4499428834496992

Pe = 2.1405351080787485



UndefVarError: UndefVarError: `Vf` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [42]:
p_SM = (machine, (V_0, θ_0, P_0, Q_0), Vf_0)

(MachineParams(3.148, 2.0, 1.79, 1.71, 0.169, 0.228, 0.135, 0.2, 0.13, 4.3, 0.85, 0.032, 0.05, 0.0, 0.002), (1.0142, -0.007843738409384567, 0.8, 0.10751144041475172), 1.8560820609476498)

In [43]:
function synchronous_machine_dynamics!(du, u, p, t)
    # Unpack parameters
    machine, pf_result, Vf = p
    (V, θ, P, Q) = pf_result

    # Unpack state variables
    δ, ω, Eq_p, Ed_p, ψq_pp, ψd_pp = u

    # Calculate terminal conditions
    # Stator voltage equations
    # TO-DO: CHECK WHETHER THE BUS ANGLE FROM POWER FLOW SOLUTION SHOULD BE INCLUDED
    Vd = V * sin(δ - θ)       # Eqn 15.4
    Vq = V * cos(δ - θ)       # Eqn 15.4
    println("Vd = $Vd\nVq = $Vq\n")

    # Terminal voltage
    Vt = sqrt(Vd^2 + Vq^2)

    # Calculate machine currents
    # Helper variables (Eqn 15.14)
    γd1 = (machine.Xdpp - machine.Xl) / (machine.Xdp - machine.Xl)
    γq1 = (machine.Xqpp - machine.Xl) / (machine.Xqp - machine.Xl)
    γd2 = (1 - γd1) / (machine.Xdp - machine.Xl)
    γq2 = (1 - γq1) / (machine.Xqp - machine.Xl)

    # Current equations (Use 15.11 and 15.15 to eliminate ψq and ψd, solve for Id and Iq)
    A = [machine.Xdpp machine.Ra; machine.Ra -machine.Xqpp]
    b = [-Vq + γd1 * Eq_p + (1 - γd1) * ψd_pp; -Vd + γq1 * Ed_p + (1 - γq1) * ψq_pp]
    currents = A \ b
    Id = currents[1]
    Iq = currents[2]
    println("Id = $Id\nIq = $Iq\n")

    # Calculate shaft conditions
    # Electromagnetic torque/power (Use 15.11 and 15.6 to eliminate ψq and ψd, solve for Te)
    Pe = Vd * Id + Vq * Iq + machine.Ra * (Id^2 + Iq^2)
    println("Pe = $Pe\n")
    # Te = Pe   # in per unit, torque equals power

    # Total mechanical power input
    Pm = Pe

    # Synchronous speed in pu
    ω_s = 1.0

    # Shaft mechanical equations (Eqn 15.5)
    du[1] = ω - ω_s  # dδ/dt
    du[2] = (Pm - Pe - machine.D * (ω - ω_s)) / (2.0 * machine.H)

    # Electrical dynamics
    du[3] = (-Eq_p - (machine.Xd - machine.Xdp) * (Id - γd2 * ψd_pp - (1 - γd1) * Id + γd2 * Eq_p) + Vf) / machine.Td0p    # dE'q/dt
    du[4] = (-Ed_p + (machine.Xq - machine.Xqp) * (Iq - γq2 * ψq_pp - (1 - γq1) * Iq - γq2 * Ed_p)) / machine.Tq0p         # dE'd/dt    TO-DO: CHECK TYPO IN γq2
    du[5] = (-ψd_pp + Eq_p - (machine.Xdp - machine.Xl) * Id) / machine.Td0pp                                              # dE''q/dt
    du[6] = (-ψq_pp - Ed_p - (machine.Xqp - machine.Xl) * Iq) / machine.Tq0pp                                              # dE''d/dt

    # Add numerical stability safeguards
    for i in 1:length(du)
        # Limit the rate of change to reasonable values
        if abs(du[i]) > 100.0
            du[i] = sign(du[i]) * 100.0
        end
    end
end

synchronous_machine_dynamics! (generic function with 1 method)

In [44]:
synchronous_machine_dynamics!(du0_SM, u0_SM, p_SM, 0)

Vd = 0.7585077083850829
Vq = 0.6732515847143696

Id = 0.6305900995301867
Iq = 2.4499428834496992

Pe = 2.1405351080787485



In [47]:
du0_SM

6-element Vector{Float64}:
  0.0
  0.0
  0.0014360192231148137
  2.4977289759401806
  0.036212201283735104
 -3.9309737270464775

In [48]:
function synchronous_machine_dynamics!(du, u, p, t)
    # Unpack parameters
    machine, pf_result = p
    (V, θ, P, Q) = pf_result

    # Unpack state variables
    δ, ω, Eq_p, Ed_p, ψq_pp, ψd_pp = u

    # Calculate terminal conditions
    # Stator voltage equations
    # TO-DO: CHECK WHETHER THE BUS ANGLE FROM POWER FLOW SOLUTION SHOULD BE INCLUDED
    Vd = V * sin(δ - θ)       # Eqn 15.4
    Vq = V * cos(δ - θ)       # Eqn 15.4
    println("Vd = $Vd\nVq = $Vq\n")

    # Terminal voltage
    Vt = sqrt(Vd^2 + Vq^2)

    # Calculate machine currents
    # Helper variables (Eqn 15.14)
    γd1 = (machine.Xdpp - machine.Xl) / (machine.Xdp - machine.Xl)
    γq1 = (machine.Xqpp - machine.Xl) / (machine.Xqp - machine.Xl)
    γd2 = (1 - γd1) / (machine.Xdp - machine.Xl)
    γq2 = (1 - γq1) / (machine.Xqp - machine.Xl)

    # Current equations (Use 15.11 and 15.15 to eliminate ψq and ψd, solve for Id and Iq)
    A = [machine.Xdpp machine.Ra; machine.Ra -machine.Xqpp]
    b = [-Vq + γd1 * Eq_p + (1 - γd1) * ψd_pp; -Vd + γq1 * Ed_p + (1 - γq1) * ψq_pp]
    currents = A \ b
    Id = currents[1]
    Iq = currents[2]
    println("Id = $Id\nIq = $Iq\n")

    # Calculate shaft conditions
    # Electromagnetic torque/power (Use 15.11 and 15.6 to eliminate ψq and ψd, solve for Te)
    Pe = Vd * Id + Vq * Iq + machine.Ra * (Id^2 + Iq^2)
    println("Pe = $Pe\n")
    # Te = Pe   # in per unit, torque equals power

    # Total mechanical power input
    Pm = Pe

    # Synchronous speed in pu
    ω_s = 1.0

    # Shaft mechanical equations (Eqn 15.5)
    du[1] = ω - ω_s  # dδ/dt
    du[2] = (Pm - Pe - machine.D * (ω - ω_s)) / (2.0 * machine.H)

    # Electrical dynamics
    du[3] = (-Eq_p - (machine.Xd - machine.Xdp) * (Id - γd2 * ψd_pp - (1 - γd1) * Id + γd2 * Eq_p) + Vf) / machine.Td0p    # dE'q/dt
    du[4] = (-Ed_p + (machine.Xq - machine.Xqp) * (Iq - γq2 * ψq_pp - (1 - γq1) * Iq - γd2 * Ed_p)) / machine.Tq0p         # dE'd/dt    TO-DO: CHECK TYPO IN γq2
    du[5] = (-ψd_pp + Eq_p - (machine.Xdp - machine.Xl) * Id) / machine.Td0pp                                              # dE''q/dt
    du[6] = (-ψq_pp - Ed_p - (machine.Xqp - machine.Xl) * Iq) / machine.Tq0pp                                              # dE''d/dt

    # Add numerical stability safeguards
    for i in 1:length(du)
        # Limit the rate of change to reasonable values
        if abs(du[i]) > 100.0
            du[i] = sign(du[i]) * 100.0
        end
    end
end

synchronous_machine_dynamics! (generic function with 1 method)

In [49]:
du0_SM = Vector{Float64}(undef, 6) # return object for derivative calculation
synchronous_machine_dynamics!(du0_SM, u0_SM, p_SM, 0)
du0_SM

Vd = 0.7585077083850829
Vq = 0.6732515847143696

Id = 0.6305900995301867
Iq = 2.4499428834496992

Pe = 2.1405351080787485



UndefVarError: UndefVarError: `Vf` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [50]:
function synchronous_machine_dynamics!(du, u, p, t)
    # Unpack parameters
    machine, pf_result, Vf = p
    (V, θ, P, Q) = pf_result

    # Unpack state variables
    δ, ω, Eq_p, Ed_p, ψq_pp, ψd_pp = u

    # Calculate terminal conditions
    # Stator voltage equations
    # TO-DO: CHECK WHETHER THE BUS ANGLE FROM POWER FLOW SOLUTION SHOULD BE INCLUDED
    Vd = V * sin(δ - θ)       # Eqn 15.4
    Vq = V * cos(δ - θ)       # Eqn 15.4
    println("Vd = $Vd\nVq = $Vq\n")

    # Terminal voltage
    Vt = sqrt(Vd^2 + Vq^2)

    # Calculate machine currents
    # Helper variables (Eqn 15.14)
    γd1 = (machine.Xdpp - machine.Xl) / (machine.Xdp - machine.Xl)
    γq1 = (machine.Xqpp - machine.Xl) / (machine.Xqp - machine.Xl)
    γd2 = (1 - γd1) / (machine.Xdp - machine.Xl)
    γq2 = (1 - γq1) / (machine.Xqp - machine.Xl)

    # Current equations (Use 15.11 and 15.15 to eliminate ψq and ψd, solve for Id and Iq)
    A = [machine.Xdpp machine.Ra; machine.Ra -machine.Xqpp]
    b = [-Vq + γd1 * Eq_p + (1 - γd1) * ψd_pp; -Vd + γq1 * Ed_p + (1 - γq1) * ψq_pp]
    currents = A \ b
    Id = currents[1]
    Iq = currents[2]
    println("Id = $Id\nIq = $Iq\n")

    # Calculate shaft conditions
    # Electromagnetic torque/power (Use 15.11 and 15.6 to eliminate ψq and ψd, solve for Te)
    Pe = Vd * Id + Vq * Iq + machine.Ra * (Id^2 + Iq^2)
    println("Pe = $Pe\n")
    # Te = Pe   # in per unit, torque equals power

    # Total mechanical power input
    Pm = Pe

    # Synchronous speed in pu
    ω_s = 1.0

    # Shaft mechanical equations (Eqn 15.5)
    du[1] = ω - ω_s  # dδ/dt
    du[2] = (Pm - Pe - machine.D * (ω - ω_s)) / (2.0 * machine.H)

    # Electrical dynamics
    du[3] = (-Eq_p - (machine.Xd - machine.Xdp) * (Id - γd2 * ψd_pp - (1 - γd1) * Id + γd2 * Eq_p) + Vf) / machine.Td0p    # dE'q/dt
    du[4] = (-Ed_p + (machine.Xq - machine.Xqp) * (Iq - γq2 * ψq_pp - (1 - γq1) * Iq - γd2 * Ed_p)) / machine.Tq0p         # dE'd/dt    TO-DO: CHECK TYPO IN γq2
    du[5] = (-ψd_pp + Eq_p - (machine.Xdp - machine.Xl) * Id) / machine.Td0pp                                              # dE''q/dt
    du[6] = (-ψq_pp - Ed_p - (machine.Xqp - machine.Xl) * Iq) / machine.Tq0pp                                              # dE''d/dt

    # Add numerical stability safeguards
    for i in 1:length(du)
        # Limit the rate of change to reasonable values
        if abs(du[i]) > 100.0
            du[i] = sign(du[i]) * 100.0
        end
    end
end

synchronous_machine_dynamics! (generic function with 1 method)

In [51]:
du0_SM = Vector{Float64}(undef, 6) # return object for derivative calculation
synchronous_machine_dynamics!(du0_SM, u0_SM, p_SM, 0)
du0_SM

Vd = 0.7585077083850829
Vq = 0.6732515847143696

Id = 0.6305900995301867
Iq = 2.4499428834496992

Pe = 2.1405351080787485



6-element Vector{Float64}:
   0.0
   0.0
   0.0014360192231148137
 -19.820220628608663
   0.036212201283735104
  -3.9309737270464775

In [56]:
Vd_0

0.7585077083850829

In [57]:
Vq_0

0.6732515847143696

In [58]:
Id_0

0.6603026749424807

In [59]:
Iq_0

0.44434404311986414

In [60]:
P_0

0.8

In [61]:
Vd_0 * Id_0 + Vq_0 * Iq_0

0.8

In [62]:
(Vq_0 * Id_0 - Vd_0 * Iq_0) - Q_0

1.3877787807814457e-16

There seems to be an error in the DQ current calculation...

In `synchronous_machine_dynamics!` we have:
```julia
A = [machine.Xdpp machine.Ra; machine.Ra -machine.Xqpp]
b = [-Vq + γd1 * Eq_p + (1 - γd1) * ψd_pp; -Vd + γq1 * Ed_p + (1 - γq1) * ψq_pp]
currents = A \ b
Id = currents[1]
Iq = currents[2]
```

After double-checking my algebra, I believe it should be:
```julia
b = [-Vq + γd1 * Eq_p + (1 - γd1) * ψd_pp; -Vd + γq1 * Ed_p - (1 - γq1) * ψq_pp]
```

Let's try calculating Id and Iq using the method in `synchronous_machine_dynamics!`, but with the error corrected:

In [63]:
#u0_SM = [δ0, 1.0, Eq_p0, Ed_p0, ψq_pp0, ψd_pp0]
#(V_0, θ_0, P_0, Q_0) = v[2], θ[2], P[2], Q[2]

Vd = V_0 * sin(δ0 - θ_0)       # Eqn 15.4
Vq = V_0 * cos(δ0 - θ_0)       # Eqn 15.4

println("Difference in Vd calculation: $(Vd - Vd_0)")
println("Difference in Vq calculation: $(Vq - Vq_0)")


Difference in Vd calculation: 0.0
Difference in Vq calculation: 0.0


In [65]:
# Helper variables (Eqn 15.14)
γd1 = (machine.Xdpp - machine.Xl) / (machine.Xdp - machine.Xl)
γq1 = (machine.Xqpp - machine.Xl) / (machine.Xqp - machine.Xl)
γd2 = (1 - γd1) / (machine.Xdp - machine.Xl)
γq2 = (1 - γq1) / (machine.Xqp - machine.Xl)

# Current equations (Use 15.11 and 15.15 to eliminate ψq and ψd, solve for Id and Iq)
A = [machine.Xdpp machine.Ra; machine.Ra -machine.Xqpp]
b = [-Vq + γd1 * Eq_p0 + (1 - γd1) * ψd_pp0; -Vd + γq1 * Ed_p0 - (1 - γq1) * ψq_pp0]
currents = A \ b
Id = currents[1]
Iq = currents[2]

println("Difference in Id calculation: $(Id - Id_0)")
println("Difference in Iq calculation: $(Iq - Iq_0)")

Difference in Id calculation: -1.1102230246251565e-16
Difference in Iq calculation: -1.6653345369377348e-16


That did it!